In [1]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import pancancer_evaluation.config as cfg
import pancancer_evaluation.utilities.analysis_utilities as au

In [2]:
results_dir = os.path.join(cfg.repo_root,
                           'cross_cancer_flip_labels',
                           'cross_cancer_flip_labels')
flip_labels_df = (
    au.load_flip_labels_results(results_dir, 'flip_labels')
      .sort_values(by=['train_identifier', 'test_identifier'])
)
print(flip_labels_df.shape)
flip_labels_df.head(n=20)

(1050, 9)


,auroc,aupr,train_identifier,test_identifier,signal,seed,data_type,percent_flip,experiment
0,1.00000,1.000000,BRAF_COAD,BRAF_COAD,signal,42,train,0.2,flip_labels
1,0.54930,0.026679,BRAF_COAD,BRAF_COAD,signal,42,test,0.2,flip_labels
2,0.58709,0.091277,BRAF_COAD,BRAF_COAD,signal,42,cv,0.2,flip_labels
0,0.99362,0.924860,BRAF_COAD,BRAF_COAD,signal,42,train,0.1,flip_labels
1,0.93182,0.077713,BRAF_COAD,BRAF_COAD,signal,42,test,0.1,flip_labels
2,0.67531,0.146030,BRAF_COAD,BRAF_COAD,signal,42,cv,0.1,flip_labels
0,0.99114,0.879190,BRAF_COAD,BRAF_COAD,shuffled,42,train,0.3,flip_labels
1,0.39716,0.019575,BRAF_COAD,BRAF_COAD,shuffled,42,test,0.3,flip_labels
2,0.60193,0.063296,BRAF_COAD,BRAF_COAD,shuffled,42,cv,0.3,flip_labels
0,1.00000,1.000000,BRAF_COAD,BRAF_COAD,shuffled,42,train,0.2,flip_labels


In [3]:
print(flip_labels_df.train_identifier.unique())

['BRAF_COAD' 'BRAF_LUAD' 'BRAF_SKCM' 'BRAF_THCA' 'EGFR_GBM' 'EGFR_LGG'
 'EGFR_LUAD' 'EGFR_SKCM' 'EGFR_UCEC' 'KRAS_COAD' 'KRAS_LUAD' 'KRAS_PAAD'
 'KRAS_READ' 'KRAS_STAD' 'KRAS_UCEC' 'PTEN_BRCA' 'PTEN_CESC' 'PTEN_COAD'
 'PTEN_GBM' 'PTEN_LUSC' 'PTEN_SKCM' 'PTEN_STAD' 'PTEN_UCEC' 'TP53_BLCA'
 'TP53_BRCA' 'TP53_CESC' 'TP53_COAD' 'TP53_ESCA' 'TP53_GBM' 'TP53_HNSC'
 'TP53_KICH' 'TP53_LGG' 'TP53_LIHC' 'TP53_LUAD' 'TP53_LUSC' 'TP53_OV'
 'TP53_PAAD' 'TP53_PRAD' 'TP53_READ' 'TP53_SARC' 'TP53_SKCM' 'TP53_STAD'
 'TP53_UCEC' 'TP53_UCS']
